# Modelagem preditiva para Análise de Crédito

## Introdução

O presente trabalho visa realizar uma modelagem preditiva, baseada nos pilares
da ciência de dados e do aprendizado de máquina. Para a análise, foi utilizada
base de dados
[default of credit card clients](https://archive.ics.uci.edu/dataset/350/default+of+credit+card+clients)
da UCI, a qual contém cerca de 30000 contas de etiquetadas de maneira binária,
os quais indicam se o cliente conseguiu (ou não) pagar a fatura mínima de
crédito dentro dos ultimos 6 meses.

## O problema de negócios

O objetivo do presente trabalho se concentra na prática da modelagem preditiva,
bem como um melhor entendimento do problema central, *a análise de crédito*.

Segundo o [Sinfab/RS](https://sinbraf.com.br/voce-sabe-o-que-e-credito-2/), crédito é:

    CRÉDITO é uma fonte adicional de recursos que não são seus, mas obtidos de terceiros (bancos, financeiras, cooperativas de crédito e outros), que possibilita a antecipação do consumo para a aquisição de bens ou contratação de serviços. 

Nesse sentido, modelar crédito é medir o quão confiável o está a vida financeira
de uma conta. Se a conta apresentar indícios de que está passando por um período
menos estável, gostariamos de evitar liberar um crédito alto demais.

## Objetivos

Desenvolver um modelo preditivo capaz de classificar a probablilidade de uma conta bancária ficar inadimplente nos próximos mês, baseado nos dados históricos coletados.

# Obetenção dos dados

## Obtendo os dados via rede

para fins de praticidade, escrevi uma pequena automação para baixar os dados
nencessários do site da UCI, descompactá-los e deixar em uma pasta a minha
disposição.

In [1]:
import os
import requests
from zipfile import ZipFile

def download_and_unzip(url, local_folder):
    if not os.path.exists(local_folder):
        os.makedirs(local_folder)

    response = requests.get(url)
    zip_file_path = os.path.join(local_folder, 'downloaded_file.zip')
    
    with open(zip_file_path, 'wb') as zip_file:
        zip_file.write(response.content)

    with ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(local_folder)

    os.remove(zip_file_path)

In [9]:
download_and_unzip(
    url='https://archive.ics.uci.edu/static/public/350/default+of+credit+card+clients.zip',
    local_folder='datasets'
)

## Visualizando uma amostra aleatória

vamos acessaros dados e checar se está tudo funcionando como o esperado antes de
iniciarmos a nossa análise.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

credit_data = pd.read_excel(
    os.path.join('datasets', 'default of credit card clients.xls'))

credit_data.sample(5)

,Unnamed: 0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X15,X16,X17,X18,X19,X20,X21,X22,X23,Y
1861,1861,20000,2,2,2,24,1,-2,-2,-2,...,0,222,6345,0,0,0,222,6345,2000,0
905,905,80000,2,2,2,40,2,0,0,0,...,62860,63976,63293,2200,2200,4000,1777,0,2000,0
21843,21843,270000,2,2,1,26,0,0,0,0,...,12014,10597,31182,1496,2273,2014,597,31182,82615,0
25910,25910,290000,2,1,1,34,-2,-2,-2,-2,...,6000,3533,880,0,1100,6000,3533,880,193,0
4895,4895,100000,2,2,1,38,1,-1,-1,-2,...,0,0,150,199,0,0,0,150,550,0


# Conhecendo o conjunto de dados

Análise exploratória

Uma vez obtidos os dados, podemos iniciar nossa análise exploratória.

In [3]:
def print_shape(data):
    rows, columns = data.shape
    print(f'O conjunto possui {columns} colunas (features) e {rows} linhas.')

print_shape(credit_data)

O conjunto possui 25 colunas (features) e 30001 linhas.


In [4]:
credit_data.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30001 entries, 0 to 30000
Data columns (total 25 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  30001 non-null  object
 1   X1          30001 non-null  object
 2   X2          30001 non-null  object
 3   X3          30001 non-null  object
 4   X4          30001 non-null  object
 5   X5          30001 non-null  object
 6   X6          30001 non-null  object
 7   X7          30001 non-null  object
 8   X8          30001 non-null  object
 9   X9          30001 non-null  object
 10  X10         30001 non-null  object
 11  X11         30001 non-null  object
 12  X12         30001 non-null  object
 13  X13         30001 non-null  object
 14  X14         30001 non-null  object
 15  X15         30001 non-null  object
 16  X16         30001 non-null  object
 17  X17         30001 non-null  object
 18  X18         30001 non-null  object
 19  X19         30001 non-null  object
 20  X20   

In [15]:
credit_data.describe()

,Unnamed: 0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X15,X16,X17,X18,X19,X20,X21,X22,X23,Y
count,30001,30001,30001,30001,30001,30001,30001,30001,30001,30001,...,30001,30001,30001,30001,30001,30001,30001,30001,30001,30001
unique,30001,82,3,8,5,57,12,12,12,12,...,21549,21011,20605,7944,7900,7519,6938,6898,6940,3
top,ID,50000,2,2,2,29,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
freq,1,3365,18112,14030,15964,1605,14737,15730,15764,16455,...,3195,3506,4020,5249,5396,5968,6408,6703,7173,23364
